In [13]:
from pymongo import MongoClient

def connect_mongo():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["Ramsaikumardb"]
    collection = db["pdf_path"]
    return collection


In [14]:
import requests
import os

def download_pdf(pdf_url, output_folder):
    try:
        response = requests.get(pdf_url)
        filename = os.path.join(output_folder, pdf_url.split('/')[-1].split('?')[0])
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    except Exception as e:
        print(f"Error downloading {pdf_url}: {e}")
        return None


In [15]:
from PyPDF2 import PdfReader

def parse_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
        return text
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def summarize_text(text):
    sentences = sent_tokenize(text)
    if len(sentences) > 5:
        return " ".join(sentences[:5])  # Taking first 5 sentences as summary for simplicity
    else:
        return text  # Return whole text if it's short

def extract_keywords(text, num_keywords=5):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    
    freq_dist = nltk.FreqDist(filtered_words)
    return [word for word, freq in freq_dist.most_common(num_keywords)]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ramsaikumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ramsaikumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def store_initial_data_in_mongo(file_path, collection):
    file_metadata = {
        "document_name": os.path.basename(file_path),
        "path": file_path,
        "size": os.path.getsize(file_path),
        "ingestion_date": datetime.utcnow(),
        "summary": None,
        "keywords": None
    }
    collection.insert_one(file_metadata)
    return file_metadata

def update_mongo_with_summary(file_metadata, summary, keywords, collection):
    query = {"path": file_metadata["path"]}
    update = {"$set": {"summary": summary, "keywords": keywords}}
    collection.update_one(query, update)


In [18]:
import json
import time

def process_pdfs_from_dataset(dataset_file, output_folder):
    # Load dataset
    with open(dataset_file, 'r') as f:
        dataset = json.load(f)
    
    # Connect to MongoDB
    collection = connect_mongo()
    
    for pdf_name, pdf_url in dataset.items():
        # Download PDF
        file_path = download_pdf(pdf_url, output_folder)
        if not file_path:
            continue  # Skip if the file couldn't be downloaded
        
        # Parse PDF
        text = parse_pdf(file_path)
        if not text:
            continue  # Skip if parsing failed
        
        # Store metadata in MongoDB
        file_metadata = store_initial_data_in_mongo(file_path, collection)
        
        # Generate summary and keywords
        summary = summarize_text(text)
        keywords = extract_keywords(text)
        
        # Update MongoDB with summary and keywords
        update_mongo_with_summary(file_metadata, summary, keywords, collection)

    print("All PDFs processed and stored.")


In [21]:
# Setting the the path to the dataset file and output folder
dataset_file = "C:\Users\Ramsaikumar\OneDrive\Desktop\wasserstof"
output_folder = "/path/to/save/downloaded/pdfs"  # Replace with an actual folder path

# Run the pipeline
process_pdfs_from_dataset(dataset_file, output_folder)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1128251287.py, line 2)

In [23]:
import os
import json
import time
from pymongo import MongoClient
import requests
from PyPDF2 import PdfReader
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from datetime import datetime

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

# MongoDB connection function
def connect_mongo():
    client = MongoClient("mongodb://localhost:27017/")
    db = client["Ramsaikumardb"]
    collection = db["pdf_path"]
    return collection

# Function to download a PDF from a URL
def download_pdf(pdf_url, output_folder):
    try:
        response = requests.get(pdf_url)
        # Extract filename from URL
        filename = os.path.join(output_folder, pdf_url.split('/')[-1].split('?')[0])
        with open(filename, 'wb') as f:
            f.write(response.content)
        return filename
    except Exception as e:
        print(f"Error downloading {pdf_url}: {e}")
        return None

# Function to parse text from a PDF file
def parse_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
        return text
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None

# Function to summarize text
def summarize_text(text):
    sentences = sent_tokenize(text)
    if len(sentences) > 5:
        return " ".join(sentences[:5])  # Taking first 5 sentences as summary
    else:
        return text  # Return whole text if it's short

# Function to extract keywords from text
def extract_keywords(text, num_keywords=5):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    freq_dist = nltk.FreqDist(filtered_words)
    return [word for word, freq in freq_dist.most_common(num_keywords)]

# Function to store initial data in MongoDB
def store_initial_data_in_mongo(file_path, collection):
    file_metadata = {
        "document_name": os.path.basename(file_path),
        "path": file_path,
        "size": os.path.getsize(file_path),
        "ingestion_date": datetime.utcnow(),
        "summary": None,
        "keywords": None
    }
    collection.insert_one(file_metadata)
    return file_metadata

# Function to update MongoDB with summary and keywords
def update_mongo_with_summary(file_metadata, summary, keywords, collection):
    query = {"path": file_metadata["path"]}
    update = {"$set": {"summary": summary, "keywords": keywords}}
    collection.update_one(query, update)

# Main function to process PDFs from dataset
def process_pdfs_from_dataset(dataset_file, output_folder):
    # Load dataset
    with open(dataset_file, 'r') as f:
        dataset = json.load(f)
    
    # Connect to MongoDB
    collection = connect_mongo()
    
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for pdf_name, pdf_url in dataset.items():
        print(f"Processing {pdf_name} from {pdf_url}")
        
        # Download PDF
        file_path = download_pdf(pdf_url, output_folder)
        if not file_path:
            continue  # Skip if the file couldn't be downloaded
        
        # Parse PDF
        text = parse_pdf(file_path)
        if not text:
            continue  # Skip if parsing failed
        
        # Store metadata in MongoDB
        file_metadata = store_initial_data_in_mongo(file_path, collection)
        
        # Generate summary and keywords
        summary = summarize_text(text)
        keywords = extract_keywords(text)
        
        # Update MongoDB with summary and keywords
        update_mongo_with_summary(file_metadata, summary, keywords, collection)
    
    print("All PDFs processed and stored.")

# Set the path to the dataset file and output folder
dataset_file = r"C:\Users\Ramsaikumar\OneDrive\Desktop\wasserstof\Dataset.json"  # Path to the JSON dataset
output_folder = r"C:\Users\Ramsaikumar\OneDrive\Desktop\wasserstof"  # Path to the output folder

# Run the pipeline
process_pdfs_from_dataset(dataset_file, output_folder)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ramsaikumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ramsaikumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing pdf1 from https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTUwL3ZvbHVtZSAxL1BhcnQgSS9Db21taXNzaW9uZXIgb2YgSW5jb21lIFRheCwgV2VzdCBCZW5nYWxfQ2FsY3V0dGEgQWdlbmN5IEx0ZC5fMTY5NzYwNjMxMC5wZGY=


C:\Users\Ramsaikumar\AppData\Local\Temp\ipykernel_7536\828506726.py:70: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ingestion_date": datetime.utcnow(),


Processing pdf2 from https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTUyL3ZvbHVtZSAxL1BhcnQgSS90aGUgc3RhdGUgb2YgYmloYXJfbWFoYXJhamFkaGlyYWphIHNpciBrYW1lc2h3YXIgc2luZ2ggb2YgZGFyYmhhbmdhIGFuZCBvdGhlcnNfMTY5ODMxODQ0OC5wZGY=
Processing pdf3 from https://cdnbbsr.s3waas.gov.in/s380537a945c7aaa788ccfcdf1b99b5d8f/uploads/2024/07/20240716890312078.pdf
Processing pdf4 from https://www.mha.gov.in/sites/default/files/250883_english_01042024.pdf
Processing pdf5 from https://rbidocs.rbi.org.in/rdocs/PressRelease/PDFs/PR60974A2ED1DFDB84EC0B3AABFB8419E1431.PDF
Processing pdf6 from https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTYwL3ZvbHVtZSAxL1BhcnQgSS90aGUgdGF0YSBvaWwgbWlsbHMgY28uIGx0ZC5faXRzIHdvcmttZW4gYW5kIG90aGVyc18xNjk5MzMzODYyLnBkZg==
Processing pdf7 from https://digiscr.sci.gov.in/pdf_viewer?dir=YWRtaW4vanVkZ2VtZW50X2ZpbGUvanVkZ2VtZW50X3BkZi8xOTYwL3ZvbHVtZSAxL1BhcnQgSS9ncmVhdCBpbmRpYW4gbW90b3Igd29ya3MgbHRkLiw